In [11]:
import pandas as pd

def select_columns(df, columns):
    df.rename(columns=columns, inplace=True)
    df.rename(columns={"Country Name": "country", "Country Code": "code", "Year": "year"}, inplace=True)
    return df[["country", "code", "year"]+[column for column in columns.values()]]

df_agriculture = pd.read_csv('./WB-datasets/agriculture-and-rural-development/agriculture-and-rural-development.csv')
agriculture_columns = {
    # "average_value_Access to electricity, rural (% of rural population)" : "porcentagem da populacao rural com acesso a eletricidade",
    "average_value_Agricultural raw materials exports (% of merchandise exports)" : "exportacao de materias primas agricolas",
    "average_value_Agricultural raw materials imports (% of merchandise imports)" : "importacao de materias primas agricolas",
    "average_value_Rural population (% of total population)" : "porcentagem da populacao rural",
    # "average_value_Agricultural methane emissions (% of total)" : "emissoes de metano agricolas",
}

df_enviroment = pd.read_csv('./WB-datasets/environment/environment.csv')
enviroment_columns = {
    # "average_value_Adjusted savings: carbon dioxide damage (% of GNI)" : "dano de CO2 ajustado",
    "average_value_Adjusted savings: education expenditure (% of GNI)" : "gastos com educacao ajustados",
    "average_value_People using at least basic drinking water services (% of population)" : "porcentagem da populacao com acesso a agua potavel",
    "average_value_People using at least basic sanitation services (% of population)" : "porcentagem da populacao com acesso a saneamento basico",
    # "average_value_Total natural resources rents (% of GDP)" : "renda total de recursos naturais",
}   

df_health = pd.read_csv('./WB-datasets/health/health.csv')
health_columns = {
    # "average_value_Population growth (annual %)" : "crescimento populacional",
    "average_value_Population, female (% of total population)" : "porcentagem da populacao feminina",
    # "average_value_Population, total" : "populacao total",
    # "average_value_Population ages 15-64 (% of total population)" : "porcentagem da populacao entre 15 e 64 anos",
}

df_poverty = pd.read_csv('./WB-datasets/poverty/poverty.csv')
poverty_columns = {
    "average_value_Gini index (World Bank estimate)" : "indice de gini",
    "average_value_Proportion of people living below 50 percent of median income (%)" : "porcentagem da populacao vivendo com menos de 50% da renda mediana",
}

df_agriculture = select_columns(df_agriculture, agriculture_columns)
df_enviroment = select_columns(df_enviroment, enviroment_columns)
df_health = select_columns(df_health, health_columns)
df_poverty = select_columns(df_poverty, poverty_columns)

#  vamos dar join nos datasets para criar o df_brabo, ele será o full outer join de todos os datasets nas colunas country, code e year
df_brabo = df_agriculture.merge(df_enviroment, on=["country", "code", "year"], how="outer")
df_brabo = df_brabo.merge(df_health, on=["country", "code", "year"], how="outer")
df_brabo = df_brabo.merge(df_poverty, on=["country", "code", "year"], how="outer")


In [12]:
countries = pd.read_json('./data/countries.json')
# lets add a continent column to the dataframe
df_brabo['continent'] = df_brabo['country'].map(countries.set_index('country')['continent'])

# Vamos mudar a entrada de country "Korea, Dem. People's Rep." para "North Korea"
df_health.loc[df_health['country'] == "Korea, Dem. People's Rep.", 'country'] = 'North Korea'
# Agora Channel Islands, que é Chile 
df_health.loc[df_health['country'] == "Channel Islands", 'country'] = 'Chile'
# Timor-Leste é East Timor
df_health.loc[df_health['country'] == "Timor-Leste", 'country'] = 'East Timor'

# vamos listar os países sem continente
df_brabo[df_brabo['continent'].isna()]['country'].unique()

# vamos remover todos os países que não tem continente
df_brabo = df_brabo[df_brabo['continent'].notna()]

# dividindo o valor da coluna do indice de gini por 100 para ficar entre 0 e 1
df_brabo['indice de gini'] = df_brabo['indice de gini'] / 100

In [13]:
# verificando os valores unicos dos países em ordem alfabetica
countries = df_brabo['country'].unique()
countries.sort()
countries.tolist()

# Agora vamos criar um dicionário com os países e seus valores corretos
country_dict = {
    'Dem. Rep. Congo': 'Congo, The Democratic Republic of the',
    'Congo, Rep.': 'Congo',
    'Egypt, Arab Rep.': 'Egypt',
    'Iran, Islamic Rep.': 'Iran',
    'Kyrgyz Republic': 'Kyrgyzstan',
    "Cote d'Ivoire": "Côte d'Ivoire",
    'Korea, Rep.': 'South Korea',
    'Lao PDR': 'Laos',
    'North Macedonia': 'Macedonia, Republic of',
    'Russian Federation': 'Russia',
    'Slovak Republic': 'Slovakia',
    'Syrian Arab Republic': 'Syria',
    'Tanzania': 'Tanzania, United Republic of',
    'United Kingdom': 'United Kingdom of Great Britain and Northern Ireland',
    'United States': 'United States of America',
    'Venezuela, RB': 'Venezuela'
}

# Agora vamos substituir os valores no dataframe
df_brabo['country'].replace(country_dict, inplace=True)

In [14]:
# vamos exportar para um csv
df_brabo.to_csv('data/data.csv', index=False)
# e agora vamos exportar para um json
df_brabo.to_json('data/data.json', orient='records')

# agora repetindo o processo para o dataset de 2018
df_brabo_2018 = df_brabo[df_brabo['year'] == 2018]
df_brabo_2018.to_csv('data/data_2018.csv', index=False)
df_brabo_2018.to_json('data/data_2018.json', orient='records')